In [7]:
import requests

In [26]:
r = requests.get("https://api.themoviedb.org/3/search/movie?api_key=8d5d54aa72351ac3e821515d968260c5&language=en-US&query=Toy%20story&page=1&include_adult=true&year=1995")

In [27]:
print(r.json()['results'][0]['poster_path'])

/uXDfjJbdP4ijW5hWSBrPrlKpxab.jpg


In [1]:
movies = db.execute('SELECT id, title, genres ''FROM movie ''EXCEPT ''SELECT m.id, m.title, m.genres ''FROM movie m ''INNER JOIN ''review r ''ON r.userID = ? ''and m.id = r.movieID '' LIMIT ? OFFSET ?',(g.user['id'], '8', str((page - 1) * 8), )).fetchall()

NameError: name 'db' is not defined

In [33]:
page=4
str((page - 1) * 8)

'24'

In [2]:

movies = ('Tom and Huck (1995)',
'GoldenEye (1995)',
'American President, The (1995)',
'Dracula: Dead and Loving It (1995)',
'Balto (1995)',
'Nixon (1995)',
'Cutthroat Island (1995)',
'Casino (1995)')
images=[]
for movie in movies:
    title = movie
    title = title.split('(')
    actual_title = title[0].rstrip()
    actual_title = actual_title.replace(' ', '&')
    data = title[1][:-1]
    r = requests.get("https://api.themoviedb.org/3/search/movie?api_key=8d5d54aa72351ac3e821515d968260c5&language=en-US&query=" + actual_title + "&page=1&include_adult=true&year=" + data)

    poster_path = r.json()['results'][0]['poster_path']
    poster_path = 'https://image.tmdb.org/t/p/w500' + poster_path
    images.append(poster_path)

listas = zip(movies, images)

NameError: name 'requests' is not defined

In [69]:
title = movie
title = title.split('(')
actual_title = title[0].rstrip()
actual_title = actual_title.replace(' ', '&')
data = title[1][:-1]


In [67]:
actual_title
actual_title = actual_title.replace(' ', '&')

In [70]:
actual_title

'American&President,&The'

In [52]:
data

<Response [200]>

In [50]:
poster_path = r.json()

In [51]:
poster_path

{'page': 1, 'results': [], 'total_pages': 0, 'total_results': 0}

In [ ]:
def recommend_movies(predictions_df,
                     userID,
                     movies_df,
                     original_ratings_df,
                     num_recommendations=5):

    # Get and sort the user's predictions
    user_row_number = userID - 1  # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(
        ascending=False)

    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.USERID == (userID)]
    user_full = (user_data.merge(
        movies_df, how='left', left_on='MOVIEID',
        right_on='MOVIEID').sort_values(['RATING'], ascending=False))

    print('User {} has already rated {} movies.'.format(userID, user_full.shape[0]))
    print('Recommending the highest {} predicted ratings movies not already rated.'.format(num_recommendations))

    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (
        movies_df[~movies_df['MOVIEID'].isin(user_full['MOVIEID'])].merge(
            pd.DataFrame(sorted_user_predictions).reset_index(),
            how='left',
            left_on='MOVIEID',
            right_on='MOVIEID').rename(columns={
                user_row_number: 'Predictions'
            }).sort_values('Predictions',
                           ascending=False).iloc[:num_recommendations, :-1])

    return user_full, recommendations

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds

def recommend_movies(predictions_df,
                     userID,
                     movies_df,
                     original_ratings_df,
                     num_recommendations=5):

    # Get and sort the user's predictions
    user_row_number = userID - 1  # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(
        ascending=False)

    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.USERID == (userID)]
    user_full = (user_data.merge(
        movies_df, how='left', left_on='MOVIEID',
        right_on='MOVIEID').sort_values(['RATING'], ascending=False))

    print('User {} has already rated {} movies.'.format(userID, user_full.shape[0]))
    print('Recommending the highest {} predicted ratings movies not already rated.'.format(num_recommendations))

    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (
        movies_df[~movies_df['MOVIEID'].isin(user_full['MOVIEID'])].merge(
            pd.DataFrame(sorted_user_predictions).reset_index(),
            how='left',
            left_on='MOVIEID',
            right_on='MOVIEID').rename(columns={
                user_row_number: 'Predictions'
            }).sort_values('Predictions',
                           ascending=False).iloc[:num_recommendations, :-1])

    return user_full, recommendations


db = sqlite3.connect("C:/Users/User10/Downloads/Movie-Recommender-master/Movie-Recommender-master/instance/flaskr.sqlite")

movie_df = pd.read_sql_query('SELECT * FROM movie', db)
rating_df = pd.read_sql_query('SELECT * FROM review', db)

# rename so they have the same stuff
#movie_df.rename({"id":"MOVIEID", "title": "TITLE", "genres": "GENRES"}, axis=1)
#rating_df.rename({"userID": "USERID", "movieID": "MOVIEID", "rating":"RATING" }, axis = 1)

movie_df.columns = ['MOVIEID', 'TITLE', 'GENRES']
rating_df.columns = ['USERID', 'MOVIEID', 'RATING']

movie_df['MOVIEID'] = movie_df['MOVIEID'].apply(pd.to_numeric)

# format matrix to be one row per user and one column per movie

R_df = rating_df.pivot(
    index='USERID', columns='MOVIEID', values='RATING').fillna(0)

# de - mean the data and convert it from a dataframe
# to a numpy array

R = R_df.values
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

# Perform singular value decomposition
# use scipys function for svds
U, sigma, Vt = svds(R_demeaned, k=50)

sigma = np.diag(sigma)

all_user_predicted_ratings = np.dot(np.dot(U, sigma),
                                    Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns=R_df.columns)

#already_rated, predictions = recommend_movies(preds_df, g.user['id'], movie_df, rating_df, 50)
already_rated, predictions = recommend_movies(preds_df, 1, movie_df, rating_df, 50)
page = 1
movies = predictions[page * 8: page * 8 + 8]

#print(predictions['GENRES'])
images = []
#r = requests.get("https://api.themoviedb.org/3/search/movie?api_key=8d5d54aa72351ac3e821515d968260c5&language=en-US&query=Toy%20story&page=1&include_adult=true&year=1995")

'''for i in range(len(movies.index)):
    title = movies.iloc[i]['TITLE']
    title = title.split('(')
    actual_title = title[0].rstrip()
    actual_title = actual_title.replace(' ', '&')
    data = title[1][:-1]
    r = requests.get("https://api.themoviedb.org/3/search/movie?api_key=8d5d54aa72351ac3e821515d968260c5&language=en-US&query=" + actual_title + "&page=1&include_adult=true&year=" + data)

    poster_path = r.json()['results'][0]['poster_path']
    poster_path = 'https://image.tmdb.org/t/p/w500' + poster_path
    images.append(poster_path)'''

User 1 has already rated 238 movies.
Recommending the highest 50 predicted ratings movies not already rated.


'for i in range(len(movies.index)):\n    title = movies.iloc[i][\'TITLE\']\n    title = title.split(\'(\')\n    actual_title = title[0].rstrip()\n    actual_title = actual_title.replace(\' \', \'&\')\n    data = title[1][:-1]\n    r = requests.get("https://api.themoviedb.org/3/search/movie?api_key=8d5d54aa72351ac3e821515d968260c5&language=en-US&query=" + actual_title + "&page=1&include_adult=true&year=" + data)\n\n    poster_path = r.json()[\'results\'][0][\'poster_path\']\n    poster_path = \'https://image.tmdb.org/t/p/w500\' + poster_path\n    images.append(poster_path)'

In [15]:
import sqlite3
import pandas as pd
con = sqlite3.connect("E:/Movie-Recommender-master/Movie-Recommender-master - Copy/instance/new.db")


In [21]:
#blog.py
test=pd.read_sql_query('SELECT m.id, m.title, m.genres, m.poster, r.rating '
                  'FROM movies m '
                  'INNER JOIN '
                  'ratings r '
                  'ON r.userID = 1 '
                  'and m.id = r.movieID '
                  'LIMIT 8 OFFSET 8', con)
test

,id,title,genres,poster,rating
0,163,Desperado (1995),Action|Romance|Western,https://image.tmdb.org/t/p/w500/e3gwpBeXpvGZsx...,5.0
1,216,Billy Madison (1995),Comedy,https://image.tmdb.org/t/p/w500/1XV42Zu9taxluI...,5.0
2,223,Clerks (1994),Comedy,https://image.tmdb.org/t/p/w500/zeFhNF0SXlYIvW...,3.0
3,231,Dumb & Dumber (Dumb and Dumber) (1994),Adventure|Comedy,https://image.tmdb.org/t/p/w500/4LdpBXiCyGKkR8...,5.0
4,235,Ed Wood (1994),Comedy|Drama,https://image.tmdb.org/t/p/w500/A5NCOebgzXe0jP...,4.0
5,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,https://image.tmdb.org/t/p/w500/6FfCtAuVAW8XJj...,5.0
6,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,https://image.tmdb.org/t/p/w500/fIE3lAGcZDV1G6...,3.0
7,316,Stargate (1994),Action|Adventure|Sci-Fi,https://image.tmdb.org/t/p/w500/5TE3obJopSD9lW...,3.0


In [25]:
images=test['poster'].to_list()

In [26]:
listas = zip(test, images)

In [29]:
listas

In [45]:
db = sqlite3.connect("E:/Movie-Recommender-master/Movie-Recommender-master - Copy/instance/new.db")
ratings = db.execute('SELECT m.id, m.title, m.genres, m.poster, r.rating '
                  'FROM movies m '
                  'INNER JOIN '
                  'ratings r '
                  'ON r.userID = 1 '
                  'and m.id = r.movieID '
                  'LIMIT 8 OFFSET 8',).fetchall()

'''images = []

for movie in ratings:
    title = movie['title']
    title = title.split('(')
    actual_title = title[0].rstrip()
    data = title[1][:-1]
    r = requests.get("https://api.themoviedb.org/3/search/movie?api_key=8d5d54aa72351ac3e821515d968260c5&language=en-US&query=" + actual_title + "&page=1&include_adult=true&year=" + data)

    poster_path = r.json()['results'][0]['poster_path']
    poster_path = 'https://image.tmdb.org/t/p/w500' + poster_path

    images.append(poster_path)

listas = zip(ratings, images)'''

'images = []\n\nfor movie in ratings:\n    title = movie[\'title\']\n    title = title.split(\'(\')\n    actual_title = title[0].rstrip()\n    data = title[1][:-1]\n    r = requests.get("https://api.themoviedb.org/3/search/movie?api_key=8d5d54aa72351ac3e821515d968260c5&language=en-US&query=" + actual_title + "&page=1&include_adult=true&year=" + data)\n\n    poster_path = r.json()[\'results\'][0][\'poster_path\']\n    poster_path = \'https://image.tmdb.org/t/p/w500\' + poster_path\n\n    images.append(poster_path)\n\nlistas = zip(ratings, images)'

In [52]:
ratings[0][3]

'https://image.tmdb.org/t/p/w500/e3gwpBeXpvGZsxUya9zNym5QXrw.jpg'

In [57]:
images = list(map(lambda poster: poster[3], ratings))

['https://image.tmdb.org/t/p/w500/e3gwpBeXpvGZsxUya9zNym5QXrw.jpg',
 'https://image.tmdb.org/t/p/w500/1XV42Zu9taxluIKXMeV9OSpYCSq.jpg',
 'https://image.tmdb.org/t/p/w500/zeFhNF0SXlYIvWXpaOLuXVQ3jB4.jpg',
 'https://image.tmdb.org/t/p/w500/4LdpBXiCyGKkR8FGHgjKlphrfUc.jpg',
 'https://image.tmdb.org/t/p/w500/A5NCOebgzXe0jPzh0mbXRWLZNr5.jpg',
 'https://image.tmdb.org/t/p/w500/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg',
 'https://image.tmdb.org/t/p/w500/fIE3lAGcZDV1G6XM5KmuWnNsPp1.jpg',
 'https://image.tmdb.org/t/p/w500/5TE3obJopSD9lWCa6rqF2E4Oda7.jpg']

In [44]:
images = []

for movie in ratings:
    title = movie['title']
    title = title.split('(')
    actual_title = title[0].rstrip()
    data = title[1][:-1]
    r = requests.get("https://api.themoviedb.org/3/search/movie?api_key=8d5d54aa72351ac3e821515d968260c5&language=en-US&query=" + actual_title + "&page=1&include_adult=true&year=" + data)

    poster_path = r.json()['results'][0]['poster_path']
    poster_path = 'https://image.tmdb.org/t/p/w500' + poster_path

    images.append(poster_path)

TypeError: tuple indices must be integers or slices, not str

In [5]:
from matplotlib import image
from sklearn.decomposition import TruncatedSVD

from scipy.sparse.linalg import svds
import sqlite3
import pandas as pd
import numpy as np
import requests

def reccomend_movies(user):
    page=1
    db = sqlite3.connect("E:/Movie-Recommender-master/Item-Based Collaborative filtering fix/instance/flaskr.sqlite")
    ratings = pd.read_sql_query('SELECT userId, movieId, rating FROM ratings',db)
    ratings.columns = ['USERID', 'MOVIEID', 'RATING']
    #Ambil data movies dari sqlite dengan querry "SELECT * FROM review" dan simpan ke pandas dataframe
    movies = pd.read_sql_query('SELECT * FROM movies', db)
    movies.columns = ['MOVIEID', 'TITLE', 'GENRES', 'POSTER']

    combined_movies_data = pd.merge(ratings, movies, how='inner', on='MOVIEID')#merge data rating dan data movie pada movieId
    #Buat pivot table untuk rating menggunakan userId sebagai index dan movieId sebagai kolom
    rating_crosstab = combined_movies_data.pivot_table(index='USERID', columns='MOVIEID', values='RATING').fillna(0)
    #lakukan transpose dan dimesionality reduction dengan menggunakan fungsi SVD ke tabel pivot yang telah dibuat
    X = rating_crosstab.T
    SVD = TruncatedSVD(n_components=12, random_state=5)
    resultant_matrix = SVD.fit_transform(X)

    #cari korelasi antar film menggunakan pearson correlation menggunakan fungsi corrcoef pada library numpy
    corr_mat = np.corrcoef(resultant_matrix)
    corr_mat = pd.DataFrame(corr_mat, columns = rating_crosstab.columns.tolist(), index = rating_crosstab.columns.tolist())
    #cari data user yang akan diberi rekomendasi berdasarkan rating film tertinggi yang diberikan 
    userId=user
    unrated=X[userId].loc[X[userId]==0]
    rated=X[userId].loc[X[userId]>0]
    #buat pivot table dengan menggunakan movieId yang sudah dirating sebagai kolom dan movieId yang belum dirating sebagai index
    r_ur_corr_matt=corr_mat[rated.index].loc[unrated.index]\
                .apply(lambda x: x.nlargest(5), axis=1).fillna(0)#cari nilai similarity pada 10 item yang sudah 

    simlilarity_dot_weight=r_ur_corr_matt.apply(lambda x: sum(x*rated), axis=1)
    simlilarity_sum=r_ur_corr_matt.apply(np.sum, axis=1)
    pred=(simlilarity_dot_weight/simlilarity_sum).sort_values(ascending=False)
    reccomendation=pd.concat([movies.set_index('MOVIEID'), pred], axis=1).sort_values(by=0, ascending=False)
    movies=reccomendation[page * 8: page * 8 + 8]
    images=movies['POSTER'].tolist()
    return reccomendation

In [4]:
from matplotlib import image
from sklearn.decomposition import TruncatedSVD

from scipy.sparse.linalg import svds
import sqlite3
import pandas as pd
import numpy as np
import requests
page=1
user=611
db = sqlite3.connect("E:/Movie-Recommender-master/Item-Based Collaborative filtering fix/instance/flaskr.sqlite")
pd.read_sql_query('SELECT * from users',db)
ratings = pd.read_sql_query('SELECT userId, movieId, rating FROM ratings',db)
ratings.columns = ['USERID', 'MOVIEID', 'RATING']
#Ambil data movies dari sqlite dengan querry "SELECT * FROM review" dan simpan ke pandas dataframe
movies = pd.read_sql_query('SELECT * FROM movies', db)
movies.columns = ['MOVIEID', 'TITLE', 'GENRES', 'POSTER']
try:
    combined_movies_data = pd.merge(ratings, movies, how='inner', on='MOVIEID')#merge data rating dan data movie pada movieId
    #Buat pivot table untuk rating menggunakan userId sebagai index dan movieId sebagai kolom
    rating_crosstab = combined_movies_data.pivot_table(index='USERID', columns='MOVIEID', values='RATING').fillna(0)
    #lakukan transpose dan dimesionality reduction dengan menggunakan fungsi SVD ke tabel pivot yang telah dibuat
    X = rating_crosstab.T
    SVD = TruncatedSVD(n_components=12, random_state=5)
    resultant_matrix = SVD.fit_transform(X)

    #cari korelasi antar film menggunakan pearson correlation menggunakan fungsi corrcoef pada library numpy
    corr_mat = np.corrcoef(resultant_matrix)
    corr_mat = pd.DataFrame(corr_mat, columns = rating_crosstab.columns.tolist(), index = rating_crosstab.columns.tolist())
    #cari data user yang akan diberi rekomendasi berdasarkan rating film tertinggi yang diberikan 
    userId=user
    unrated=X[userId].loc[X[userId]==0]
    rated=X[userId].loc[X[userId]>0]
except:
    reccomendation=movies


In [5]:
reccomendation

,MOVIEID,TITLE,GENRES,POSTER
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://image.tmdb.org/t/p/w500/uXDfjJbdP4ijW5...
1,2,Jumanji (1995),Adventure|Children|Fantasy,https://image.tmdb.org/t/p/w500/z46R8oShx61gXM...
2,3,Grumpier Old Men (1995),Comedy|Romance,https://image.tmdb.org/t/p/w500/1FSXpj5e8l4KH6...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,https://image.tmdb.org/t/p/w500/4wjGMwPsdlvi02...
4,5,Father of the Bride Part II (1995),Comedy,https://image.tmdb.org/t/p/w500/rj4LBtwQ0uGrpB...
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,https://image.tmdb.org/t/p/w500/4jU2Bdk1MB2OvP...
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,https://image.tmdb.org/t/p/w500/cCBB6BGRj5nCTa...
9739,193585,Flint (2017),Drama,https://image.tmdb.org/t/p/w500/iPzkjNWpK7ud5p...
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,https://image.tmdb.org/t/p/w500/7ueB7Kxq2GObp5...


In [7]:
rated

MOVIEID
1         5.0
6         5.0
16        4.5
32        4.5
47        5.0
         ... 
166534    4.0
168248    5.0
168250    5.0
168252    5.0
170875    3.0
Name: 610, Length: 1302, dtype: float64

In [6]:
movies1=recommended()
movies1

User 1 has already rated 238 movies.
Recommending the highest 50 predicted ratings movies not already rated.


,MOVIEID,TITLE,GENRES
2759,4011,Snatch (2000),Comedy|Crime|Thriller
1411,2081,"Little Mermaid, The (1989)",Animation|Children|Comedy|Musical|Romance
825,1200,Aliens (1986),Action|Adventure|Horror|Sci-Fi
1415,2085,101 Dalmatians (One Hundred and One Dalmatians...,Adventure|Animation|Children
469,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi
1102,1610,"Hunt for Red October, The (1990)",Action|Adventure|Thriller
767,1101,Top Gun (1986),Action|Romance
983,1407,Scream (1996),Comedy|Horror|Mystery|Thriller


In [9]:
movies1.iloc[3]['MOVIEID']

2085

In [2]:
def recommended():
    page = request.args.get('page', 1, type=int)

    if page is None:
        page = 1

    db = get_db()

    #Ambil data ratings dari sqlite dengan querry "SELECT * FROM review" dan simpan ke pandas dataframe
    ratings = pd.read_sql_query('SELECT userId, movieId, rating FROM ratings',db)
    ratings.columns = ['USERID', 'MOVIEID', 'RATING']
    #Ambil data movies dari sqlite dengan querry "SELECT * FROM review" dan simpan ke pandas dataframe
    movies = pd.read_sql_query('SELECT * FROM movies', db)
    movies.columns = ['MOVIEID', 'TITLE', 'GENRES', 'POSTER']

    combined_movies_data = pd.merge(ratings, movies, how='inner', on='MOVIEID')#merge data rating dan data movie pada movieId
    #Buat pivot table untuk rating menggunakan userId sebagai index dan movieId sebagai kolom
    rating_crosstab = combined_movies_data.pivot_table(index='USERID', columns='MOVIEID', values='RATING').fillna(0)
    #lakukan transpose dan dimesionality reduction dengan menggunakan fungsi SVD ke tabel pivot yang telah dibuat
    X = rating_crosstab.T
    SVD = TruncatedSVD(n_components=12, random_state=5)
    resultant_matrix = SVD.fit_transform(X)

    #cari korelasi antar film menggunakan pearson correlation menggunakan fungsi corrcoef pada library numpy
    corr_mat = np.corrcoef(resultant_matrix)
    corr_mat = pd.DataFrame(corr_mat, columns = rating_crosstab.columns.tolist(), index = rating_crosstab.columns.tolist())
    #cari data user yang akan diberi rekomendasi berdasarkan rating film tertinggi yang diberikan 
    userId=5
    unrated=X[userId].loc[X[userId]==0]
    rated=X[userId].loc[X[userId]>0]
    #buat pivot table dengan menggunakan movieId yang sudah dirating sebagai kolom dan movieId yang belum dirating sebagai index
    r_ur_corr_matt=corr_mat[rated.index].loc[unrated.index]\
                .apply(lambda x: x.nlargest(5), axis=1).fillna(0)#cari nilai similarity pada 10 item yang sudah 

    simlilarity_dot_weight=r_ur_corr_matt.apply(lambda x: sum(x*rated), axis=1)
    simlilarity_sum=r_ur_corr_matt.apply(np.sum, axis=1)
    pred=(simlilarity_dot_weight/simlilarity_sum).sort_values(ascending=False)
    reccomendation=pd.concat([movies.set_index('MOVIEID'), pred], axis=1).sort_values(by=0, ascending=False)
    movies=reccomendation[page * 8: page * 8 + 8]
    images=movies['POSTER'].tolist()